#### SIADS 696 Milestone II

In [ ]:
import pandas as pd
from data_columns import childhood, demographic, diagnosis


In [12]:
df = pd.read_csv('./data/mental_health_dataframe.csv')
df.shape

(5001, 1000)